In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

## GET INDEKS LINK

In [ ]:
def getIndeksLink(links, page, cat_link, category, date=datetime.strftime(datetime.today(), '%Y-%m-%d')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : YYYY-mm-dd
    """
    print("page ", page)
    if cat_link == 'otomotif':
        url = "https://indeks.kompas.com/"+cat_link+"/"+date+"/"+str(page)
    else :
        url = "https://indeks.kompas.com/"+cat_link+"/"+date+"/"+str(page)
    print(url)
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
    contentDiv = soup.findAll('div', class_="latest--indeks mt2 clearfix")
    for post in contentDiv:
        link = [post.find('a', href=True)['href'], category]
        links.append(link)
        
    el_page = soup.find('div', class_="paging__wrap clearfix")
    if el_page:
        a_page = soup.find('div', class_="paging__wrap clearfix").findAll('div', class_='paging__item')[-1].find('a')
        if 'data-ci-pagination-page' in a_page:
            max_page = page
        else:
            max_page = int(a_page['data-ci-pagination-page'].replace('\n', '').strip(' '))
            
        if page < max_page:
            links = getIndeksLink(links, page+1, cat_link, category, date)
        
    return links

## GET DETAIL BERITA

In [198]:
def getDetailBerita(links):
    all_articles = []
    for link in links:
        articles = {}
        #link
        url = link[0]
        response = requests.get(url)
        html = response.text
        
        # Create a BeautifulSoup object from the HTML: soup
        soup = BeautifulSoup(html, "html5lib")
        
        #extract subcategory from breadcrumb
        bc = soup.find('div', class_="breadcrumbs")
        if not bc:
            continue
        
        sub = bc.findAll('a')[1].text
        if ("foto" in sub.lower()) or  "video" in sub.lower(): 
            continue
            
        articles['subcategory'] = sub
        
        #articles['id'] = int(soup.find("meta", attrs={'name':'articleid'})['content'])
        
        #category
        articles['category'] = link[1]
        articles['url'] = url
        
        article = soup.find('div', class_="tru")
        
        #extract date
#         pubdate = soup.find("meta", attrs={'name':'publishdate'})['content']
#         pubdate = pubdate.strip(' \t\n\r')
#         articles['pubdate'] = datetime.strftime(datetime.strptime(pubdate, "%Y/%m/%d %H:%M:%S"), '%Y-%m-%d %H:%M:%S')
        
        #extract author
#         articles['author'] = soup.find("meta", attrs={'name':'author'})['content']
        
        #extract title
        articles['title'] = soup.find('meta', attrs={"property":"og:title"})['content']
        
        #source
        articles['source'] = 'metrotvnews'
        
        #extract comments count
        #articles['comments'] = int(soup.find('a', class_="komentar").find('span').text.replace('Komentar', '').strip(' \t\n\r'))
        
        #extract tags
        tags = soup.find('div', class_="line").findAll('a', class_="tag")
        articles['tags'] = ','.join([x.text for x in tags])
        
        #extract images
        articles['images'] = soup.find('img', class_="pic")['src']
        
        #extract detail
        detail = soup.find('div', class_="tru")
        
        #hapus link sisip
        for link in detail.findAll('div', class_="related extra"):
            link.decompose()
        
#         #hapus video sisip
#         for tag in detail.findAll('div', class_="sisip_embed_sosmed"):
#             tag.decompose()
            
#         #hapus all setelah clear fix
#         for det in detail.find('div', class_="clearfix mb20").findAllNext():
#             det.decompose()
            
#         #hapus all script
#         for script in detail.findAll('script'):
#             script.decompose()
        
        #extract content
#         detail = BeautifulSoup(detail.decode_contents().replace('<br/>', ' '), "html5lib")
#         content = re.sub(r'\n|\t|\b|\r','',detail.text)
#         articles['content'] = re.sub(r'(Tonton juga).*','', content)
#         print('memasukkan berita id ', articles['id'])
#         all_articles.append(articles)
    return all_articles

In [194]:
response = requests.get("http://otomotif.metrotvnews.com/modifikasi/GNlAap2b-the-black-bison-glc-class-dari-tanah-tiongkok")
    # Extract HTML texts contained in Response object: html
html = response.text
    # Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html, "html5lib")

In [5]:
# articles['id'] = int(soup.find("meta", attrs={'name':'cXenseParse:articleid'})['content'])

In [48]:
articles = {}
bc = soup.find('div', class_="breadcrumbs")
articles['subcategory'] = bc.findAll('a')[1].text
articles['subcategory']

'Modifikasi'

In [172]:
article = soup.find("div", class_="page")
# article

In [174]:
pubdate = soup.find("div",class_="reg").text
pubdate = pubdate.strip(' \t\n\r')
pubdate = pubdate
pubdate
# pubdate = pubdate.replace()
# articles['pubdate'] = datetime.strftime(datetime.strptime(pubdate, "%Y/%m/%d %H:%M:%S"), '%Y-%m-%d %H:%M:%S')

'M. Bagus Rachmanto \xa0\xa0 • \xa0\xa0 Sabtu, 28 Jul 2018 12:25 WIB'

In [121]:
nonBreakSpace = '\xa0'
pubdate = pubdate.replace(nonBreakSpace, '')

In [124]:
pubdate = [pubdate]
pubdate

[['M. Bagus Rachmanto  •  Sabtu, 28 Jul 2018 12:25 WIB']]

In [127]:
pubdate[0]

['M. Bagus Rachmanto  •  Sabtu, 28 Jul 2018 12:25 WIB']

In [177]:
articles['title'] = soup.find('meta', attrs={"property":"og:title"})['content']
articles['title']

'The Black Bison GLC-Class dari Tanah Tiongkok'

In [143]:
# articles['comments'] = int(soup.find('a', class_="komentar").find('span').text.replace('Komentar', '').strip(' \t\n\r'))
# articles['comments'] 

In [179]:
tags = soup.find('div', class_="line").findAll('a', class_="tag")
articles['tags'] = ','.join([x.text for x in tags])
articles['tags']

'modifikasi mobil'

In [181]:
articles['images'] = soup.find('img', class_="pic")['src']
articles['images']

'http://cdn.metrotvnews.com/dynamic/content/2018/07/28/907819/Zgx2Qm96lt.jpg?w=650'

In [ ]:
detail = article.find('div', class_="detail_text")

In [187]:
detail = soup.find('div', class_="tru")
detail

<div class="tru">
            <p>
            <strong>Jepang: </strong>Tuner asal Jepang, Wald International, punya cara tersendiri dalam melakukan modifikasi dan memiliki tampilan khusus yang berlaku untuk semua <em>body kit</em>-nya, sehigga sangat mudah diprediksi kalau itu adalah hasil garapanya.<br/><br/>
Hal itu terlihat dari hasil modifikasi yang mereka lakukan terhadapp SUV Mercedes pertama mereka, yakni dengan mengubah tampilan SUV asal Jerman itu seperti GLC-Class dan dijuluki The Black Bison GLC-Class.<br/>
<img alt="" class="big_center fluid" src="http://cdn.medcom.id/images/library/images/oto/2018/07%20Juli/GLC3.jpg"/><br/>
Ciri khas modifikasi hasil karya Wald International terlihat pada bagian bumper baru, dengan  memasangkan kisi-kisi yang direvisi dan spoiler dagu yang terinspirasi F1 yang dipasang di bagian tengah. 
 

    </p><div class="related">
        <div class="ti3 ani cl-effect-4">
            <a href="http://otomotif.metrotvnews.com/topic/5808">Baca juga</a>


In [188]:
#hapus link sisip
for link in detail.findAll('div', class_="related extra"):
    link.decompose()

In [190]:
links = "http://otomotif.metrotvnews.com/modifikasi/GNlAap2b-the-black-bison-glc-class-dari-tanah-tiongkok"

In [200]:
# getDetailBerita("http://otomotif.metrotvnews.com/modifikasi/GNlAap2b-the-black-bison-glc-class-dari-tanah-tiongkok")